In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np

In [4]:
ds = load_dataset("ckandemir/bitcoin_tweets_sentiment_kaggle")
print(ds)

README.md:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

c:\Users\James\anaconda3\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\James\.cache\huggingface\hub\datasets--ckandemir--bitcoin_tweets_sentiment_kaggle. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


(…)-00000-of-00001-cc8461398e266567.parquet:   0%|          | 0.00/7.92M [00:00<?, ?B/s]

(…)-00000-of-00001-922aa10406034550.parquet:   0%|          | 0.00/997k [00:00<?, ?B/s]

(…)-00000-of-00001-dc793d916ae447cb.parquet:   0%|          | 0.00/949k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/77791 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9724 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/9724 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Date', 'text', 'Sentiment'],
        num_rows: 77791
    })
    test: Dataset({
        features: ['Date', 'text', 'Sentiment'],
        num_rows: 9724
    })
    eval: Dataset({
        features: ['Date', 'text', 'Sentiment'],
        num_rows: 9724
    })
})


In [ ]:
s = "Most popular trading pair is ONE/USDT on Binance.Theory: Individual MMs prefer the smaller effective tick size of ONE/USDT, as opposed to the ONE/BTC pair (due to decimal precision)"

print(len(s))

181


: 

In [5]:
ds = ds.filter(lambda example:example["Sentiment"] in ["Positive", "Negative"])


Filter:   0%|          | 0/77791 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9724 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9724 [00:00<?, ? examples/s]

In [6]:
label2id = {"Negative": 0, "Positive": 1}
id2label = {0: "Negative", 1: "Positive"}

def encode_label(example):
    return {"labels": label2id[example["Sentiment"]]}

ds = ds.map(encode_label)

Map:   0%|          | 0/77788 [00:00<?, ? examples/s]

Map:   0%|          | 0/9724 [00:00<?, ? examples/s]

Map:   0%|          | 0/9724 [00:00<?, ? examples/s]

In [7]:
model_name = "vinai/bertweet-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# For binary classification, set num_labels=2
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)


c:\Users\James\anaconda3\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

c:\Users\James\anaconda3\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\James\.cache\huggingface\hub\models--vinai--bertweet-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

PackageNotFoundError: bitsandbytes

In [ ]:
def tokenize_fn(examples):
  return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

ds = ds.map(tokenize_fn, batched=True)

print(ds)

ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



In [ ]:
ds["train"][0]

train_ds = ds["train"]
eval_ds = ds["eval"]
test_ds = ds["test"]




In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"]

    return {
        "accuracy": acc,
        "f1": f1
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="bertweet-bitcoin-sentiment",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

# ------------------------------------------------------------------------------
# 8. Evaluate on test set
# ------------------------------------------------------------------------------
results = trainer.evaluate(test_ds)
print("Test set results:", results)